In [1]:
import sys, os

sys.path.append(os.path.abspath(".."))
from configs.general import GeneralConfig
import numpy as np
import pandas as pd

In [2]:
elemental_abundances = (
    GeneralConfig.initial_abundances @ GeneralConfig.stoichiometric_matrix
)

elements = ["H", "HE", "C", "N", "O", "S", "SI", "MG", "CL"]
def build_molecular_matrix():
    S = GeneralConfig.stoichiometric_matrix
    M = S.copy()
    for j, sp in enumerate(GeneralConfig.species):
        if sp in elements:
            e_idx = elements.index(sp)
            M[j, e_idx] = 0
    return M

In [3]:
ATOMIC_SPECIES = [sp for sp in elements if sp in GeneralConfig.species]
ATOMIC_IDX = np.array([GeneralConfig.species.index(sp) for sp in elements])
ELEMENT_IDX = np.array([elements.index(sp) for sp in ATOMIC_SPECIES])
MOLECULAR_MATRIX = build_molecular_matrix()

np.save("molecular_matrix.npy", MOLECULAR_MATRIX)

def rebuild_atomic_abundances(initial_abundances, elemental_abundances):
    """
    Works for multiple rows of initial_abundances.
    Each row is treated as a separate system of species abundances.
    """
    x = initial_abundances.copy()

    atoms = elemental_abundances - (x @ MOLECULAR_MATRIX)

    x[:, ATOMIC_IDX] = atoms[:, ELEMENT_IDX]

    return x

z = GeneralConfig.initial_abundances.copy()
z[0][ATOMIC_IDX] = 1e-30
init_abunds = pd.DataFrame(z, columns=GeneralConfig.species)
init_abunds = pd.concat([init_abunds, init_abunds])


init_abunds[elements]

,H,HE,C,N,O,S,SI,MG,CL
0,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30
0,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30


In [39]:
init_df = pd.DataFrame(GeneralConfig.initial_abundances, columns=GeneralConfig.species)
init_df = pd.concat([init_df, init_df])
init_df[elements]

,H,HE,C,N,O,S,SI,MG,CL
0,0.5,0.1,1.000000e-10,0.000062,0.000334,1.000000e-30,1.000000e-30,0.000002,1.000000e-30
0,0.5,0.1,1.000000e-10,0.000062,0.000334,1.000000e-30,1.000000e-30,0.000002,1.000000e-30


In [40]:
test = rebuild_atomic_abundances(init_abunds.values, elemental_abundances)

test_df = pd.DataFrame(test, columns=GeneralConfig.species)

test_df[elements]

,H,HE,C,N,O,S,SI,MG,CL
0,0.5,0.1,1.000000e-10,0.000062,0.000334,0.0,0.0,0.000002,0.0
1,0.5,0.1,1.000000e-10,0.000062,0.000334,0.0,0.0,0.000002,0.0


In [12]:
df = pd.read_hdf("../data/grav_collapse_clean.h5", key="val")
df

,Index,Model,Time,Density,Radfield,Av,gasTemp,#C,#C2,#C2H,...,SIH5+,SIO,SIO+,SIOH+,SIS,SIS+,SO,SO+,SO2,SO2+
0,594,2.0,0.0,2.471193e+06,0.000100,122.304202,13.316765,1.000000e-30,1.000000e-30,1.000000e-30,...,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30,1.000000e-30
1,595,2.0,92.9,2.471245e+06,0.000893,119.686260,37.272744,2.813550e-13,6.056061e-11,1.919478e-15,...,3.909999e-17,3.848474e-07,1.549187e-13,3.260137e-10,4.504716e-11,3.829415e-17,1.557687e-13,1.539742e-14,1.127069e-17,4.943645e-26
2,596,2.0,185.8,2.471405e+06,0.002462,120.643656,37.273202,1.330024e-13,2.803234e-10,2.961034e-13,...,2.443958e-13,5.780698e-07,1.466270e-14,1.820768e-09,1.443427e-10,5.267001e-18,1.121715e-12,3.376170e-12,1.724015e-17,4.188084e-26
3,597,2.0,278.7,2.471670e+06,0.004076,131.574973,37.273966,1.781387e-13,3.408449e-10,4.671095e-13,...,2.950991e-13,5.895586e-07,8.385838e-15,2.892375e-09,1.891399e-10,6.995355e-18,1.890156e-12,3.770928e-12,2.900173e-17,6.765509e-26
4,598,2.0,371.6,2.472039e+06,0.006141,132.060001,37.275035,2.387678e-13,4.145124e-10,7.088585e-13,...,2.952988e-13,5.804863e-07,5.375523e-15,3.499727e-09,2.113931e-10,9.369751e-18,2.328810e-12,3.792042e-12,3.572433e-17,8.825884e-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889213,2963758,9978.0,27126.8,7.763906e+04,0.004172,4.395878,28.220272,1.993537e-14,9.905272e-10,1.823899e-11,...,7.428627e-14,3.570409e-08,3.374260e-16,2.323992e-10,2.944902e-12,1.366942e-14,6.448945e-12,1.881876e-11,2.166335e-14,2.056398e-20
889214,2963759,9978.0,27219.7,7.710364e+04,0.004286,2.907921,28.196477,1.911747e-14,9.899456e-10,1.826711e-11,...,7.393906e-14,3.563371e-08,3.354582e-16,2.311452e-10,2.911839e-12,1.357824e-14,6.466022e-12,1.876441e-11,2.158776e-14,2.064687e-20
889215,2963760,9978.0,27312.6,7.657411e+04,0.009786,1.173676,28.172518,1.839867e-14,9.893626e-10,1.829151e-11,...,7.357795e-14,3.556663e-08,3.423802e-16,2.304969e-10,2.879267e-12,1.348634e-14,6.482933e-12,1.865966e-11,2.151327e-14,2.073020e-20
889216,2963761,9978.0,27405.5,7.605042e+04,0.005952,5.029348,28.148750,2.276693e-14,9.887758e-10,1.832739e-11,...,7.232625e-14,3.574368e-08,7.269153e-16,2.357076e-10,2.853023e-12,1.333271e-14,6.486427e-12,2.005257e-11,2.141177e-14,2.135194e-20


In [42]:
df1 = df[(df["Model"] == 2) & (df["Time"] == 27405.5)][GeneralConfig.species]


np1 = df1.to_numpy()

elem_abund = (
    np1 @ GeneralConfig.stoichiometric_matrix
)
print(elem_abund)

print(reconstruct_atoms(elem_abund, np1, M))
print(f"{df1['O'].iloc[0]:.3e}")

[[9.99999527e-01 1.00000000e-01 1.77000100e-04 6.18000000e-05
  3.34000000e-04 3.51000000e-06 1.78000000e-06 2.25600000e-06
  3.39000000e-08]]
[[9.02354413e-10 1.00000000e-01 4.05404903e-13 5.30654248e-05
  1.09202241e-06 8.10747296e-07 2.42960961e-15 2.20236112e-06
  1.29445101e-14]]
1.092e-06
